# Продвинутое машинное обучение: ДЗ 3
### Герасимчик Анна. ML-22

## Описание

Третье домашнее задание посвящено достаточно простой, но, надеюсь, интересной задаче, в которой потребуется творчески применить методы сэмплирования. Как и раньше, в качестве решения ожидается ссылка на jupyter-ноутбук на вашем github (или публичный, или с доступом для snikolenko); ссылку обязательно нужно прислать в виде сданного домашнего задания на портале Академии. Как всегда, любые комментарии, новые идеи и рассуждения на тему категорически приветствуются. 

В этом небольшом домашнем задании мы попробуем улучшить метод Шерлока Холмса. Как известно, в рассказе The Adventure of the Dancing Men великий сыщик расшифровал загадочные письмена. Пользовался он для этого так называемым частотным методом: смотрел, какие буквы чаще встречаются в зашифрованных текстах, и пытался подставить буквы в соответствии с частотной таблицей: E — самая частая и так далее.

В этом задании мы будем разрабатывать более современный и продвинутый вариант такого частотного метода. В качестве корпусов текстов для подсчётов частот можете взять что угодно, но для удобства вот вам $\href{https://www.dropbox.com/s/k23enjvr3fb40o5/corpora.zip}{“Война~и~мир”}$ по-русски и по-английски:

In [1]:
import os
import re
import random
import numpy as np
from copy import copy
from tqdm import tqdm
from typing import Dict, List
from collections import Counter

In [2]:
DATA_PATH = 'corpora/'
RUSSIAN_ALPHABET = ' абвгдеёжзийклмнопрстуфхцчшщъыьэюя'
SAMPLE_LENGTH = 600

In [3]:
def read_data(path):
    with open(path, encoding="utf8") as f:
        data = f.read().replace('\n', ' ')
        return data

In [4]:
anna_karenina = read_data(DATA_PATH + 'AnnaKarenina.txt')
war_and_peace = read_data(DATA_PATH + 'WarAndPeace.txt')

### 1

Реализуйте базовый частотный метод по Шерлоку Холмсу:
- подсчитайте частоты букв по корпусам (пунктуацию и капитализацию можно просто опустить, а вот пробелы лучше оставить);
- возьмите какие-нибудь тестовые тексты (нужно взять по меньшей мере 2-3 предложения, иначе вряд ли сработает), зашифруйте их посредством случайной перестановки символов;
- расшифруйте их таким частотным методом.

In [5]:
def clean_text(text: str, alphabet: str) -> str:
    cleaned = ''
    for c in text.lower():
        if c in alphabet:
            cleaned += c
    return cleaned


def accuracy(original: str, decoded: str) -> float:
    correct = 0
    for i in range(len(original)):
        if original[i] == decoded[i]:
            correct += 1
    return correct / len(original)


def get_freq_dict(text: str, step: int = 1) -> Dict:
    freq_dict = Counter()
    for i in range(0, len(text) - 1):
        freq_dict[text[i:i + step]] += 1
    freq_dict_sorted = sorted(dict(freq_dict).items(), key=lambda item: item[1], reverse=True)
    freq_dict_balanced = {}
    for k, v in freq_dict_sorted:
        freq_dict_balanced[k] = v / len(text)
    return freq_dict_balanced

In [6]:
def encode(text: str, step: int = 1) -> str:
    encode_dict = get_encode_dict(text, step)
    encoded = ''
    for i in range(len(text) - step + 1):
        encoded += encode_dict[text[i:i + step]]
    return encoded


def get_encode_dict(text: str, step:int = 1) -> Dict:
    freq_dict = get_freq_dict(text, step)
    keys = list(freq_dict.keys())
    shuffled_keys = keys.copy()
    random.shuffle(shuffled_keys)
    encode_dict = {}
    for i in range(len(shuffled_keys)):
        encode_dict[keys[i]] = shuffled_keys[i]
    return encode_dict

In [7]:
def decode(text: str, freq_dict: Dict, step: int = 1) -> Dict:
    decode_dict = get_decode_dict(text, freq_dict, step)
    decoded = ''
    for i in range(len(text) - step + 1):
        if text[i:i + step] in decode_dict.keys():
            decoded += decode_dict[text[i:i + step]]
        else:
            decoded += str('*' * step)
    return decoded


def get_decode_dict(text: str, freq_list: List, step: int = 1) -> Dict:
    freq_dict = get_freq_dict(text, step)
    keys = list(freq_dict.keys())
    decode_dict = {}
    for i in range(min(len(freq_dict),len(freq_list))):
        decode_dict[keys[i]] = freq_list[i]
    return decode_dict

In [8]:
def decoding(train: str, freq_dict: Dict, step: int = 1) -> str:
    encoded = encode(train, step)
    decoded = decode(encoded, list(freq_dict.keys()), step)
    decoding_accuracy = accuracy(train, decoded)
    print(f'sample: {train[:SAMPLE_LENGTH]}\n')
    print(f'encoded: {encoded[:SAMPLE_LENGTH]}\n')
    print(f'decoded: {decoded[:SAMPLE_LENGTH]}\n')
    print(f'accuracy: {round(decoding_accuracy * 100, 3)}%')
    return decoded

In [9]:
anna_karenina = clean_text(anna_karenina, RUSSIAN_ALPHABET)
war_and_peace = clean_text(war_and_peace, RUSSIAN_ALPHABET)

In [10]:
anna_karenina_dict = get_freq_dict(anna_karenina)
war_and_peace_dict = get_freq_dict(war_and_peace)

In [11]:
print(f'anna_karenina_dict: {anna_karenina_dict}')
print(f'war_and_peace_dict: {war_and_peace_dict}')

anna_karenina_dict: {' ': 0.17910812333922474, 'о': 0.0944538757383889, 'е': 0.0719124047007973, 'а': 0.06810538002492653, 'н': 0.05707570954251148, 'и': 0.05459526954211601, 'т': 0.049224375426371064, 'с': 0.04369063882525431, 'л': 0.041242185741628294, 'в': 0.03871114825470658, 'р': 0.03273657378247617, 'к': 0.028183381575419626, 'д': 0.024212351253567268, 'м': 0.023569705016653553, 'у': 0.022175075445505052, 'п': 0.019827817334931518, 'я': 0.017705630802354006, 'ь': 0.016198756232359942, 'ы': 0.015245546112630005, 'г': 0.014942542773777474, 'б': 0.014375501976500666, 'ч': 0.013875342914287277, 'з': 0.013446718229576498, 'ж': 0.009316916484486636, 'й': 0.008643446491413258, 'ш': 0.007018511119524639, 'х': 0.006389241229623607, 'ю': 0.00512488564677255, 'э': 0.0029189515502895396, 'щ': 0.002357726556061724, 'ц': 0.0023222501574628675, 'ф': 0.0010357945230256367, 'ъ': 0.00023961108561850773, 'ё': 1.802898945187801e-05}
war_and_peace_dict: {' ': 0.17873405534032794, 'о': 0.0933591911165

In [12]:
decoded = decoding(anna_karenina, anna_karenina_dict)

sample:    анна каренина один из самых знаменитых романов льва толстого начинается ставшей афоризмом фразой все счастливые семьи похожи друг на друга каждая несчастливая семья несчастлива посвоему это книга о вечных ценностях о любви о вере о семье о человеческом достоинстве            лев толстойроман широкого дыхания  часть первая   лев толстой  анна каренина     роман широкого дыхания     анна каренина поразила современников вседневностью содержания необычайная свобода раскованность повествования удивительно сочетались в этом романе с цельностью художественного взгляда автора на жизнь он выступал з

encoded:    оппо долбпыпо уяып ыю еожнэ юпожбпыцнэ лужопук мрко цумецуёу посыпобцев ецокщба оиулыюжуж илоюуа кеб есоецмыкнб ебжры ъуэузы яльё по яльёо дозяов пбесоецмыков ебжрв пбесоецмыко ъуекубжь йцу дпыёо у кбспнэ тбппуецвэ у мчгкы у кблб у ебжрб у сбмукбсбедуж яуецуыпецкб            мбк цумецуалужоп щылудуёу янэопыв  соецр ъблков   мбк цумецуа  оппо долбпыпо     лужоп щылудуёу янэопы

In [13]:
decoded = decoding(war_and_peace, anna_karenina_dict)

sample:    война и мир  самый известный роман льва николаевича толстого как никакое другое произведение писателя отражает глубину его мироощущения и философииэта книга из разряда вечных потому что она обо всем  о жизни и смерти о любви и чести о мужестве и героизме о славе и подвиге о войне и мирепервый том знакомит с высшим обществом россии  века показаны взаимоотношения между родителями и детьми в семье ростовых сватовство у болконских интриги у безуховых вечера в салоне фрейлины апшерер балы в москве и петербурге            лев николаевич толстойчасть первая  часть орая  часть третья              л

encoded: ссслвымцсасеарссицефысаэлъигмфысрвецмсшблцсмаьвшцълащцсгвшигвувсьцьсмаьцьвъсхр увъстрваэлъхъмаъстаицгъшюсвгрцзцъгсуш йам съувсеарввк къмаюсасёашвивёаангцсьмауцсаэсрцэрюхцслъщмфжствгве сщгвсвмцсвйвслиъессвсзаэмасасиеъргасвсшчйласасщъигасвсе зъиглъсасуърваэеъсвсишцлъсаствхлауъсвслвымъсасеарътърлфысгвесэмцьвеагсислфидаесвйкъиглвесрвииаасслъьцствьцэцмфслэцаеввгмвдъмаюсеъзх срвхагъшю

### 2

Вряд ли в результате получилась такая уж хорошая расшифровка, разве что если вы брали в качестве тестовых данных целые рассказы. Но и Шерлок Холмс был не так уж прост: после буквы E, которая действительно выделяется частотой, дальше он анализировал уже конкретные слова и пытался угадать, какими они могли бы быть. Я не знаю, как запрограммировать такой интуитивный анализ, так что давайте просто сделаем следующий логический шаг:
- подсчитайте частоты биграмм (т.е. пар последовательных букв) по корпусам;
- проведите тестирование аналогично п.1, но при помощи биграмм.

In [14]:
decoded = decoding(war_and_peace, anna_karenina_dict, step=2)

sample:    война и мир  самый известный роман льва николаевича толстого как никакое другое произведение писателя отражает глубину его мироощущения и философииэта книга из разряда вечных потому что она обо всем  о жизни и смерти о любви и чести о мужестве и героизме о славе и подвиге о войне и мирепервый том знакомит с высшим обществом россии  века показаны взаимоотношения между родителями и детьми в семье ростовых сватовство у болконских интриги у безуховых вечера в салоне фрейлины апшерер балы в москве и петербурге            лев николаевич толстойчасть первая  часть орая  часть третья              л

encoded: ититбьутгнятщььфхлзмеоцемакнитайюгзцьдяюцвхлтсыщящей пыеуаяюцвщеу ыбжднщвцэтипчкдшьфа дзксззымоцж ьнщркогльфчлббымез пббэмсбпфмнсржгжча дзкссржгззегэ ршбыожепсбегэ отф у еятсыщящ лч эгдзбхэ отщиётюгрлячефпизчлез  ядпяхнрж эрниавбчеаншлэпндфедппкхсбпфеоцемау иьаецкекелэгдзфизчхлзмйоскйбгтмюоэчьск вьюжжпцьфмниддзджвньфхлтсржщедпядынысинжеьфбьящмсгыуаюлбюотщэз ббыбагеднеыцббпфлепыщ

### 3

Но и это ещё не всё: биграммы скорее всего тоже далеко не всегда работают. Основная часть задания — в том, как можно их улучшить:
- предложите метод обучения перестановки символов в этом задании, основанный на MCMC-сэмплировании, но по-прежнему работающий на основе статистики биграмм;
- реализуйте и протестируйте его, убедитесь, что результаты улучшились.

In [21]:
def mcmc(text, text_for_train, iters, alphabet):
    
    def get_log_likelihood(text, permutation):
        text = text.translate(str.maketrans(alphabet, ''.join(permutation)))
        likelihood = 0
        for i in range(len(text) - 1):
            likelihood += transitions_matrix[embeddings[text[i]], embeddings[text[i+1]]]
        return likelihood


    embeddings = {c: i for i, c in enumerate(RUSSIAN_ALPHABET)}
    transitions_matrix = np.zeros((len(embeddings), len(embeddings)))
    for i in range(len(text_for_train)-1):
        x = embeddings[text_for_train[i]]
        y = embeddings[text_for_train[i + 1]]
        transitions_matrix[x, y] += 1
    transitions_matrix = np.clip(transitions_matrix, 1, None)
    transitions_matrix = (np.log(transitions_matrix).T - np.log(transitions_matrix.sum(axis=1))).T
    
    permutation = np.array(list(RUSSIAN_ALPHABET))
    random.shuffle(permutation)
    permutation_best = permutation.copy()
    
    log_likelihood = get_log_likelihood(text, permutation)
    log_likelihood_best = log_likelihood
    
    for i in tqdm(range(iters)):
        swapped = random.sample(range(len(RUSSIAN_ALPHABET)), 2)
        permutation[swapped[0]], permutation[swapped[1]] = permutation[swapped[1]], permutation[swapped[0]]
        
        log_likelihood_current = get_log_likelihood(text, permutation)
        if log_likelihood_current < log_likelihood:
            delta = np.exp(log_likelihood_current - log_likelihood)
            if delta < random.random():
                permutation[swapped[0]], permutation[swapped[1]] = permutation[swapped[1]], permutation[swapped[0]]
            else:
                log_likelihood = log_likelihood_current
        else:
            log_likelihood = log_likelihood_current
            if log_likelihood_current > log_likelihood_best:
                permutation_best = permutation.copy()
                log_likelihood_best = log_likelihood_current
    return text.translate(str.maketrans(RUSSIAN_ALPHABET, ''.join(permutation_best)))

In [16]:
encoded = encode(anna_karenina)[:SAMPLE_LENGTH]
decoded = mcmc(encoded, war_and_peace, 100000, RUSSIAN_ALPHABET)
decoding_accuracy = accuracy(anna_karenina[:SAMPLE_LENGTH], decoded)

100%|████████████████████████████████████████████████████████████████████████| 100000/100000 [00:31<00:00, 3129.31it/s]


In [17]:
print(f'encoded: {encoded[:SAMPLE_LENGTH]}\n')
print(f'decoded: {decoded[:SAMPLE_LENGTH]}\n')
print(f'accuracy: {round(decoding_accuracy * 100, 3)}%')

encoded: ооохммхотхлъмщмхордщмощыоюхйсзоымхйъмщесзолрйхмржоёфжхоерёюераромхящмхъеюгоюехжиъуохцрлщыйрйоцлхыруожюъоюяхюеёщжсъоюъйфщоврзршщодлнаомходлнахотхшдхгомъюяхюеёщжхгоюъйфгомъюяхюеёщжховрюжръйноперотмщахорожъямсзоэъммрюегзороёк жщорожълъороюъйфъорояъёржъяъютрйодрюерщмюежъооооооооооооёъжоерёюерулрйхмоищлртрародсзхмщгоояхюефовължхгоооёъжоерёюеруоохммхотхлъмщмхооооолрйхмоищлртрародсзхмщгооооохммхотхлъмщмховрлхыщёхоюржлъйъммщтржожюъдмъжмрюефкоюрдълшхмщгомър сяхумхгоюжр рдхолхютржхммрюефовржъюежржхмщгондщжщеъёфмроюряъехёщюфожоперйолрйхмъоюоэъёфмрюефкозндршъюежъммрарожыаёгдхохжерлхомхошщымформожсюенвхёоы

decoded:    анна каренина один из самых знаменитых романов льва толстого начинается ставшей афоризмом фразой все счастливые семьи похожи друг на друга каждая несчастливая семья несчастлива посвоему это книга о вечных ценностях о любви о вере о семье о человеческом достоинстве            лев толстойроман широкого дыхания  часть первая   лев толстой  анна каренина     роман широкого дыхан

### 4

Расшифруйте сообщение:

←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏

Или это (они одинаковые, второй вариант просто на случай проблем с юникодом):

დჳჵჂႨშႼႨშჂხჂჲდႨსႹႭჾႣჵისႼჰႨჂჵჂႨႲႹႧჲჂႨსႹႭჾႣჵისႼჰႨჲდႩჳჲႨჇႨႠჲႹქႹႨჳႹႹჱჶდსჂႽႨႩႹჲႹႭႼჰႨჵდქႩႹႨႲႭႹႧჂჲႣჲიႨჳႩႹႭდდႨშჳდქႹႨშႼႨშჳდႨჳხდჵႣჵჂႨႲႭႣშჂჵისႹႨჂႨႲႹჵჇႧჂჲდႨჾႣႩჳჂჾႣჵისႼჰႨჱႣჵჵႨეႣႨႲႹჳჵდხსდდႨႧდჲშდႭჲႹდႨეႣხႣსჂდႨႩჇႭჳႣႨႾႹჲႽႨႩႹსდႧსႹႨႽႨსჂႧდქႹႨსდႨႹჱდჶႣნ


In [24]:
def get_ru_letters(freq_dict: Dict) -> List:
    ru_letters = []
    for i in list(freq_dict.keys()):
        if i in RUSSIAN_ALPHABET:
            ru_letters.append(i)
    return ru_letters

In [18]:
sample = 'დჳჵჂႨშႼႨშჂხჂჲდႨსႹႭჾႣჵისႼჰႨჂჵჂႨႲႹႧჲჂႨსႹႭჾႣჵისႼჰႨჲდႩჳჲႨჇႨႠჲႹქႹႨჳႹႹჱჶდსჂႽႨႩႹჲႹႭႼჰႨჵდქႩႹႨႲႭႹႧჂჲႣჲიႨჳႩႹႭდდႨშჳდქႹႨშႼႨშჳდႨჳხდჵႣჵჂႨႲႭႣშჂჵისႹႨჂႨႲႹჵჇႧჂჲდႨჾႣႩჳჂჾႣჵისႼჰႨჱႣჵჵႨეႣႨႲႹჳჵდხსდდႨႧდჲშდႭჲႹდႨეႣხႣსჂდႨႩჇႭჳႣႨႾႹჲႽႨႩႹსდႧსႹႨႽႨსჂႧდქႹႨსდႨႹჱდჶႣნ'
sample = sample.translate(str.maketrans(''.join(set(sample)), ''.join(get_ru_letters(war_and_peace_dict)[:len(set(sample))])))
decoded = mcmc(sample, war_and_peace + anna_karenina, 1000000, RUSSIAN_ALPHABET)

100%|██████████████████████████████████████████████████████████████████████| 1000000/1000000 [02:26<00:00, 6837.62it/s]


In [19]:
print(f'decoded: {decoded}\n')

decoded: если вы вимите нордальный или почти нордальный текст у этого сообщения который легко прочитать скорее всего вы все смелали правильно и получите даксидальный балл за послемнее четвертое замание курса хотя конечно я ничего не обещаж

